In [ ]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 基础配置信息
n_splits = 5
seed = 2018

# lgb 参数
params={
    "num_leaves":120,
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":11,
    "silent":True,
}

# 读取数据
train = pd.read_csv('train/train_all.csv')
test = pd.read_csv('test/republish_test.csv')

'''
简单分析数据：
user_id 为编码后的数据，大小：
train data shape (612652, 27)
train data of user_id shape 612652
简单的1个用户1条样本的题目,标签的范围 current_service
'''
print('标签',set(train.columns)-set(test.columns))

print('train data shape',train.shape)
print('train data of user_id shape',len(set(train['user_id'])))
print('train data of current_service shape',(set(train['current_service'])))

print('train data shape',test.shape)
print('train data of user_id shape',len(set(test['user_id'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)

def filln(data):
    data = data.replace('\N', -1)
    data.gender = data.gender.astype(int)
    data.age = data.age.astype(int)
    data['3_total_fee'] = data['3_total_fee'].astype(float)
    data['2_total_fee'] = data['2_total_fee'].astype(float)
    return data

def trans_1(data):    
    data.service_type = data.service_type.replace(4,0)
    data.service_type = data.service_type.replace(3,0)
    
#     contract_type 3->2  9->3  12->4  7->5  2->7
    data.contract_type = data.contract_type.replace(2,10)
    data.contract_type = data.contract_type.replace(3,2)
    data.contract_type = data.contract_type.replace(9,3)
    data.contract_type = data.contract_type.replace(12,4)
    data.contract_type = data.contract_type.replace(7,5)
    data.contract_type = data.contract_type.replace(10,7)
    
    data.net_service = data.net_service.replace(9,1)
    
    data = data.loc[data.gender != -1]
    return data

def one_hot(data):
    data1 = data
#     for i in range(0,2):
#         data2 = data1.loc[data1.service_type == i,['user_id', 'service_type']]
#         data2 = data2.groupby('user_id')['service_type'].size().reset_index().rename(columns = {'service_type':'stype_' + str(i)})
#         data = pd.merge(data,data2,on = 'user_id', how = 'left')

    for i in range(0,2):
        data2 = data1.loc[data1.is_mix_service == i]
        data2 = data2.groupby('user_id')['is_mix_service'].size().reset_index().rename(columns = {'is_mix_service':'ismix_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,2):
        data2 = data1.loc[data1.many_over_bill == i]
        data2 = data2.groupby('user_id')['many_over_bill'].size().reset_index().rename(columns = {'many_over_bill':'mob_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,9):
        data2 = data1.loc[data1.contract_type == i]
        data2 = data2.groupby('user_id')['contract_type'].size().reset_index().rename(columns = {'contract_type':'ctype_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,2):
        data2 = data1.loc[data1.is_promise_low_consume == i]
        data2 = data2.groupby('user_id')['is_promise_low_consume'].size().reset_index().rename(columns = {'is_promise_low_consume':'islow_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
#     for i in range(1,5):
#         data2 = data1.loc[data1.net_service == i]
#         data2 = data2.groupby('user_id')['net_service'].size().reset_index().rename(columns = {'net_service':'ns_' + str(i)})
#         data = pd.merge(data,data2,on = 'user_id', how = 'left')    
    
    for i in range(0,3):
        data2 = data1.loc[data1.gender == i]
        data2 = data2.groupby('user_id')['gender'].size().reset_index().rename(columns = {'gender':'gender_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
    return data

def predata(data):
    #最近几月总金额
    data['total_fee'] = data['1_total_fee'] + data['2_total_fee'] + data['3_total_fee'] + data['4_total_fee']
    data['1_2_total_fee'] = data['1_total_fee'] + data['2_total_fee']
    
    #连续出账月数
    data['fee_month'] = 0
    data.loc[data['1_total_fee'] >0]['fee_month'] += 1
    data.loc[data['2_total_fee'] >0]['fee_month'] += 1
    data.loc[data['3_total_fee'] >0]['fee_month'] += 1
    data.loc[data['4_total_fee'] >0]['fee_month'] += 1
    
    #总通话时长
    data['call_time'] = data['local_caller_time'] + data['service1_caller_time'] + data['service2_caller_time']
    
    #总流量
    data['traffic'] = data['last_month_traffic'] + data['local_trafffic_month']
    
    
    return data

train = filln(train)
train = trans_1(train)
train = one_hot(train)
# train = predata(train)
test = filln(test)
test = trans_1(test)
test = one_hot(test)
# test = predata(test)

# 构造原始数据
y = train.pop('current_service')
train_id = train.pop('user_id')
# 这个字段有点问题
X = train
train_col = train.columns

X_test = test[train_col]
test_id = test['user_id']

# 数据有问题数据
# for i in train_col:
#     X[i] = X[i].replace("\\N",-1)
#     X_test[i] = X_test[i].replace("\\N",-1)

X,y,X_test = X.values,y,X_test.values

# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(11, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)
    print train_data
    clf=lgb.train(params,train_data,num_boost_round=100000,valid_sets=[validation_data]
                  ,early_stopping_rounds=50,
                  feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv('sub/sub.csv',index=False)

print(xx_score,np.mean(xx_score))

('\xe6\xa0\x87\xe7\xad\xbe', set(['current_service']))
('train data shape', (743990, 27))
('train data of user_id shape', 743990)
('train data of current_service shape', set([99999830, 90063345, 90155946, 99999825, 99999826, 99999827, 99999828, 89950166, 89950167, 89950168, 90109916]))
('train data shape', (200000, 26))
('train data of user_id shape', 200000)
0
[1]	valid_0's multi_logloss: 2.03303	valid_0's f1_score: 0.805354
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 1.80293	valid_0's f1_score: 0.813904
[3]	valid_0's multi_logloss: 1.62873	valid_0's f1_score: 0.820886
[4]	valid_0's multi_logloss: 1.49117	valid_0's f1_score: 0.830959
[5]	valid_0's multi_logloss: 1.37557	valid_0's f1_score: 0.833136
[6]	valid_0's multi_logloss: 1.27462	valid_0's f1_score: 0.834468
[7]	valid_0's multi_logloss: 1.18751	valid_0's f1_score: 0.838587
[8]	valid_0's multi_logloss: 1.11245	valid_0's f1_score: 0.841842
[9]	valid_0's multi_logloss: 1.04559	valid_0's

[115]	valid_0's multi_logloss: 0.26347	valid_0's f1_score: 0.90668
[116]	valid_0's multi_logloss: 0.262914	valid_0's f1_score: 0.906832
[117]	valid_0's multi_logloss: 0.262276	valid_0's f1_score: 0.906927
[118]	valid_0's multi_logloss: 0.261553	valid_0's f1_score: 0.907237
[119]	valid_0's multi_logloss: 0.260966	valid_0's f1_score: 0.907387
[120]	valid_0's multi_logloss: 0.260369	valid_0's f1_score: 0.907731
[121]	valid_0's multi_logloss: 0.259759	valid_0's f1_score: 0.907955
[122]	valid_0's multi_logloss: 0.25928	valid_0's f1_score: 0.908123
[123]	valid_0's multi_logloss: 0.258824	valid_0's f1_score: 0.908339
[124]	valid_0's multi_logloss: 0.258097	valid_0's f1_score: 0.908517
[125]	valid_0's multi_logloss: 0.257593	valid_0's f1_score: 0.908604
[126]	valid_0's multi_logloss: 0.256947	valid_0's f1_score: 0.908864
[127]	valid_0's multi_logloss: 0.256401	valid_0's f1_score: 0.909337
[128]	valid_0's multi_logloss: 0.255851	valid_0's f1_score: 0.909557
[129]	valid_0's multi_logloss: 0.2553

[235]	valid_0's multi_logloss: 0.224838	valid_0's f1_score: 0.918827
[236]	valid_0's multi_logloss: 0.22467	valid_0's f1_score: 0.918867
[237]	valid_0's multi_logloss: 0.224468	valid_0's f1_score: 0.918928
[238]	valid_0's multi_logloss: 0.224295	valid_0's f1_score: 0.919036
[239]	valid_0's multi_logloss: 0.224092	valid_0's f1_score: 0.919132
[240]	valid_0's multi_logloss: 0.223955	valid_0's f1_score: 0.91911
[241]	valid_0's multi_logloss: 0.223783	valid_0's f1_score: 0.919114
[242]	valid_0's multi_logloss: 0.223639	valid_0's f1_score: 0.919188
[243]	valid_0's multi_logloss: 0.223455	valid_0's f1_score: 0.919275
[244]	valid_0's multi_logloss: 0.223277	valid_0's f1_score: 0.91924
[245]	valid_0's multi_logloss: 0.22313	valid_0's f1_score: 0.919341
[246]	valid_0's multi_logloss: 0.222966	valid_0's f1_score: 0.91941
[247]	valid_0's multi_logloss: 0.22281	valid_0's f1_score: 0.919414
[248]	valid_0's multi_logloss: 0.222635	valid_0's f1_score: 0.919494
[249]	valid_0's multi_logloss: 0.22247	v

[355]	valid_0's multi_logloss: 0.209949	valid_0's f1_score: 0.923136
[356]	valid_0's multi_logloss: 0.209865	valid_0's f1_score: 0.923176
[357]	valid_0's multi_logloss: 0.209766	valid_0's f1_score: 0.923194
[358]	valid_0's multi_logloss: 0.209696	valid_0's f1_score: 0.923202
[359]	valid_0's multi_logloss: 0.209632	valid_0's f1_score: 0.923238
[360]	valid_0's multi_logloss: 0.209551	valid_0's f1_score: 0.923276
[361]	valid_0's multi_logloss: 0.209431	valid_0's f1_score: 0.923323
[362]	valid_0's multi_logloss: 0.209312	valid_0's f1_score: 0.923432
[363]	valid_0's multi_logloss: 0.209259	valid_0's f1_score: 0.923503
[364]	valid_0's multi_logloss: 0.209185	valid_0's f1_score: 0.923501
[365]	valid_0's multi_logloss: 0.209085	valid_0's f1_score: 0.923488
[366]	valid_0's multi_logloss: 0.209023	valid_0's f1_score: 0.923476
[367]	valid_0's multi_logloss: 0.208881	valid_0's f1_score: 0.923527
[368]	valid_0's multi_logloss: 0.208785	valid_0's f1_score: 0.92358
[369]	valid_0's multi_logloss: 0.20

[475]	valid_0's multi_logloss: 0.201027	valid_0's f1_score: 0.925981
[476]	valid_0's multi_logloss: 0.200939	valid_0's f1_score: 0.926001
[477]	valid_0's multi_logloss: 0.200878	valid_0's f1_score: 0.925968
[478]	valid_0's multi_logloss: 0.200836	valid_0's f1_score: 0.925972
[479]	valid_0's multi_logloss: 0.200777	valid_0's f1_score: 0.926018
[480]	valid_0's multi_logloss: 0.200717	valid_0's f1_score: 0.926031
[481]	valid_0's multi_logloss: 0.200625	valid_0's f1_score: 0.926107
[482]	valid_0's multi_logloss: 0.200562	valid_0's f1_score: 0.92612
[483]	valid_0's multi_logloss: 0.200501	valid_0's f1_score: 0.926126
[484]	valid_0's multi_logloss: 0.200448	valid_0's f1_score: 0.926213
[485]	valid_0's multi_logloss: 0.200412	valid_0's f1_score: 0.926235
[486]	valid_0's multi_logloss: 0.200331	valid_0's f1_score: 0.926214
[487]	valid_0's multi_logloss: 0.20029	valid_0's f1_score: 0.926201
[488]	valid_0's multi_logloss: 0.200237	valid_0's f1_score: 0.92626
[489]	valid_0's multi_logloss: 0.2001

...

[643]	valid_0's multi_logloss: 0.19204	valid_0's f1_score: 0.929606
[644]	valid_0's multi_logloss: 0.192004	valid_0's f1_score: 0.929584
[645]	valid_0's multi_logloss: 0.191974	valid_0's f1_score: 0.929612
[646]	valid_0's multi_logloss: 0.191934	valid_0's f1_score: 0.929654
[647]	valid_0's multi_logloss: 0.19189	valid_0's f1_score: 0.929694
[648]	valid_0's multi_logloss: 0.191852	valid_0's f1_score: 0.929694
[649]	valid_0's multi_logloss: 0.191845	valid_0's f1_score: 0.929687
[650]	valid_0's multi_logloss: 0.19181	valid_0's f1_score: 0.929683
[651]	valid_0's multi_logloss: 0.191739	valid_0's f1_score: 0.929684
[652]	valid_0's multi_logloss: 0.191674	valid_0's f1_score: 0.929701
[653]	valid_0's multi_logloss: 0.191632	valid_0's f1_score: 0.929722
[654]	valid_0's multi_logloss: 0.191597	valid_0's f1_score: 0.929729
[655]	valid_0's multi_logloss: 0.19154	valid_0's f1_score: 0.929782
[656]	valid_0's multi_logloss: 0.191503	valid_0's f1_score: 0.929749
[657]	valid_0's multi_logloss: 0.19145

[764]	valid_0's multi_logloss: 0.18813	valid_0's f1_score: 0.931117
[765]	valid_0's multi_logloss: 0.188106	valid_0's f1_score: 0.931104
[766]	valid_0's multi_logloss: 0.188076	valid_0's f1_score: 0.931111
[767]	valid_0's multi_logloss: 0.188032	valid_0's f1_score: 0.931154
[768]	valid_0's multi_logloss: 0.187996	valid_0's f1_score: 0.931216
[769]	valid_0's multi_logloss: 0.187976	valid_0's f1_score: 0.931208
[770]	valid_0's multi_logloss: 0.18794	valid_0's f1_score: 0.931237
[771]	valid_0's multi_logloss: 0.187922	valid_0's f1_score: 0.931221
[772]	valid_0's multi_logloss: 0.187895	valid_0's f1_score: 0.931262
[773]	valid_0's multi_logloss: 0.187872	valid_0's f1_score: 0.931243
[774]	valid_0's multi_logloss: 0.187836	valid_0's f1_score: 0.931256
[775]	valid_0's multi_logloss: 0.187813	valid_0's f1_score: 0.931255
[776]	valid_0's multi_logloss: 0.187783	valid_0's f1_score: 0.931227
[777]	valid_0's multi_logloss: 0.187754	valid_0's f1_score: 0.931242
[778]	valid_0's multi_logloss: 0.187

[884]	valid_0's multi_logloss: 0.185133	valid_0's f1_score: 0.931884
[885]	valid_0's multi_logloss: 0.185098	valid_0's f1_score: 0.931916
[886]	valid_0's multi_logloss: 0.185071	valid_0's f1_score: 0.931901
[887]	valid_0's multi_logloss: 0.185049	valid_0's f1_score: 0.931915
[888]	valid_0's multi_logloss: 0.185038	valid_0's f1_score: 0.931909
[889]	valid_0's multi_logloss: 0.185017	valid_0's f1_score: 0.931907
[890]	valid_0's multi_logloss: 0.184997	valid_0's f1_score: 0.931933
[891]	valid_0's multi_logloss: 0.184976	valid_0's f1_score: 0.931931
[892]	valid_0's multi_logloss: 0.184951	valid_0's f1_score: 0.931911
[893]	valid_0's multi_logloss: 0.184938	valid_0's f1_score: 0.931905
[894]	valid_0's multi_logloss: 0.184914	valid_0's f1_score: 0.931905
[895]	valid_0's multi_logloss: 0.184887	valid_0's f1_score: 0.931939
[896]	valid_0's multi_logloss: 0.184868	valid_0's f1_score: 0.931919
[897]	valid_0's multi_logloss: 0.184851	valid_0's f1_score: 0.93197
[898]	valid_0's multi_logloss: 0.18

[1003]	valid_0's multi_logloss: 0.182642	valid_0's f1_score: 0.932672
[1004]	valid_0's multi_logloss: 0.182627	valid_0's f1_score: 0.932702
[1005]	valid_0's multi_logloss: 0.182616	valid_0's f1_score: 0.932679
[1006]	valid_0's multi_logloss: 0.182607	valid_0's f1_score: 0.932694
[1007]	valid_0's multi_logloss: 0.182593	valid_0's f1_score: 0.93272
[1008]	valid_0's multi_logloss: 0.182568	valid_0's f1_score: 0.932726
[1009]	valid_0's multi_logloss: 0.182561	valid_0's f1_score: 0.932706
[1010]	valid_0's multi_logloss: 0.182534	valid_0's f1_score: 0.932691
[1011]	valid_0's multi_logloss: 0.18251	valid_0's f1_score: 0.932712
[1012]	valid_0's multi_logloss: 0.182481	valid_0's f1_score: 0.932717
[1013]	valid_0's multi_logloss: 0.18246	valid_0's f1_score: 0.932775
[1014]	valid_0's multi_logloss: 0.182447	valid_0's f1_score: 0.93277
[1015]	valid_0's multi_logloss: 0.182442	valid_0's f1_score: 0.932782
[1016]	valid_0's multi_logloss: 0.182429	valid_0's f1_score: 0.932791
[1017]	valid_0's multi_l

[1121]	valid_0's multi_logloss: 0.180606	valid_0's f1_score: 0.933437
[1122]	valid_0's multi_logloss: 0.180592	valid_0's f1_score: 0.933428
[1123]	valid_0's multi_logloss: 0.180581	valid_0's f1_score: 0.933442
[1124]	valid_0's multi_logloss: 0.18057	valid_0's f1_score: 0.933467
[1125]	valid_0's multi_logloss: 0.180552	valid_0's f1_score: 0.933382
[1126]	valid_0's multi_logloss: 0.180532	valid_0's f1_score: 0.933381
[1127]	valid_0's multi_logloss: 0.180517	valid_0's f1_score: 0.933381
[1128]	valid_0's multi_logloss: 0.180501	valid_0's f1_score: 0.933454
[1129]	valid_0's multi_logloss: 0.180485	valid_0's f1_score: 0.933423
[1130]	valid_0's multi_logloss: 0.180477	valid_0's f1_score: 0.933382
[1131]	valid_0's multi_logloss: 0.180458	valid_0's f1_score: 0.933435
[1132]	valid_0's multi_logloss: 0.18045	valid_0's f1_score: 0.933427
[1133]	valid_0's multi_logloss: 0.180434	valid_0's f1_score: 0.933456
[1134]	valid_0's multi_logloss: 0.180409	valid_0's f1_score: 0.933483
[1135]	valid_0's multi

In [3]:
df_test.predict.value_counts()

90063345    115980
89950167     20419
89950166     17187
90109916      9899
99999828      8728
90155946      6939
99999826      5413
99999827      4298
99999830      4124
89950168      3786
99999825      3227
Name: predict, dtype: int64

In [17]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 基础配置信息
n_splits = 5
seed = 42

# lgb 参数
params={
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":3,
    "silent":True,
}

# 读取数据
train = pd.read_csv('train/train_all.csv')
test = pd.read_csv('test/republish_test.csv')

train = train.loc[train.service_type == 1].reset_index(drop = True)
test = test.loc[test.service_type == 1].reset_index(drop = True)

'''
简单分析数据：
user_id 为编码后的数据，大小：
train data shape (612652, 27)
train data of user_id shape 612652
简单的1个用户1条样本的题目,标签的范围 current_service
'''
print('标签',set(train.columns)-set(test.columns))

print('train data shape',train.shape)
print('train data of user_id shape',len(set(train['user_id'])))
print('train data of current_service shape',(set(train['current_service'])))

print('train data shape',test.shape)
print('train data of user_id shape',len(set(test['user_id'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)

# 构造原始数据
y = train.pop('current_service')
train_id = train.pop('user_id')
# 这个字段有点问题
X = train
train_col = train.columns

X_test = test[train_col]
test_id = test['user_id']

# 数据有问题数据
for i in train_col:
    X[i] = X[i].replace("\\N",-1)
    X_test[i] = X_test[i].replace("\\N",-1)

X,y,X_test = X.values,y,X_test.values

# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)
#     print train_data.get_label().value_counts()
#     print validation_data.get_label().value_counts()
    clf=lgb.train(params,train_data,num_boost_round=100000,
                  valid_sets=[validation_data],early_stopping_rounds=50,
                  feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv('sub/sub1.csv',index=False)

print(xx_score,np.mean(xx_score))

('\xe6\xa0\x87\xe7\xad\xbe', set(['current_service']))
('train data shape', (347352, 27))
('train data of user_id shape', 347352)
('train data of current_service shape', set([90063345, 90155946, 90109916]))
('train data shape', (132818, 26))
('train data of user_id shape', 132818)
0
[1]	valid_0's multi_logloss: 0.965364	valid_0's f1_score: 0.978058
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 0.854822	valid_0's f1_score: 0.977265
[3]	valid_0's multi_logloss: 0.760763	valid_0's f1_score: 0.977489
[4]	valid_0's multi_logloss: 0.68088	valid_0's f1_score: 0.977776
[5]	valid_0's multi_logloss: 0.611287	valid_0's f1_score: 0.977841
[6]	valid_0's multi_logloss: 0.550958	valid_0's f1_score: 0.978056
[7]	valid_0's multi_logloss: 0.497807	valid_0's f1_score: 0.9783
[8]	valid_0's multi_logloss: 0.451217	valid_0's f1_score: 0.978476
[9]	valid_0's multi_logloss: 0.409271	valid_0's f1_score: 0.97871
[10]	valid_0's multi_logloss: 0.372183	valid_0's f1_sco

[116]	valid_0's multi_logloss: 0.0170214	valid_0's f1_score: 0.993954
[117]	valid_0's multi_logloss: 0.0169509	valid_0's f1_score: 0.993968
[118]	valid_0's multi_logloss: 0.0168712	valid_0's f1_score: 0.993997
[119]	valid_0's multi_logloss: 0.0168086	valid_0's f1_score: 0.994011
[120]	valid_0's multi_logloss: 0.0167253	valid_0's f1_score: 0.994054
[121]	valid_0's multi_logloss: 0.0166398	valid_0's f1_score: 0.994055
[122]	valid_0's multi_logloss: 0.0165935	valid_0's f1_score: 0.994069
[123]	valid_0's multi_logloss: 0.0165162	valid_0's f1_score: 0.994085
[124]	valid_0's multi_logloss: 0.0164561	valid_0's f1_score: 0.994085
[125]	valid_0's multi_logloss: 0.0163931	valid_0's f1_score: 0.994113
[126]	valid_0's multi_logloss: 0.0163382	valid_0's f1_score: 0.994156
[127]	valid_0's multi_logloss: 0.0162693	valid_0's f1_score: 0.99417
[128]	valid_0's multi_logloss: 0.0162045	valid_0's f1_score: 0.99417
[129]	valid_0's multi_logloss: 0.0161565	valid_0's f1_score: 0.994199
[130]	valid_0's multi_

[235]	valid_0's multi_logloss: 0.0132186	valid_0's f1_score: 0.994979
[236]	valid_0's multi_logloss: 0.0132103	valid_0's f1_score: 0.994979
[237]	valid_0's multi_logloss: 0.0132003	valid_0's f1_score: 0.994964
[238]	valid_0's multi_logloss: 0.0131973	valid_0's f1_score: 0.994949
[239]	valid_0's multi_logloss: 0.0131752	valid_0's f1_score: 0.994993
[240]	valid_0's multi_logloss: 0.0131493	valid_0's f1_score: 0.994964
[241]	valid_0's multi_logloss: 0.0131357	valid_0's f1_score: 0.994993
[242]	valid_0's multi_logloss: 0.0131274	valid_0's f1_score: 0.994993
[243]	valid_0's multi_logloss: 0.0131096	valid_0's f1_score: 0.994978
[244]	valid_0's multi_logloss: 0.0130879	valid_0's f1_score: 0.99495
[245]	valid_0's multi_logloss: 0.0130747	valid_0's f1_score: 0.994978
[246]	valid_0's multi_logloss: 0.0130661	valid_0's f1_score: 0.994964
[247]	valid_0's multi_logloss: 0.0130568	valid_0's f1_score: 0.994964
[248]	valid_0's multi_logloss: 0.0130566	valid_0's f1_score: 0.994978
[249]	valid_0's multi

1
[1]	valid_0's multi_logloss: 0.96516	valid_0's f1_score: 0.979522
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 0.854401	valid_0's f1_score: 0.978744
[3]	valid_0's multi_logloss: 0.760306	valid_0's f1_score: 0.978941
[4]	valid_0's multi_logloss: 0.680277	valid_0's f1_score: 0.979071
[5]	valid_0's multi_logloss: 0.610655	valid_0's f1_score: 0.979262
[6]	valid_0's multi_logloss: 0.550257	valid_0's f1_score: 0.979406
[7]	valid_0's multi_logloss: 0.49714	valid_0's f1_score: 0.979643
[8]	valid_0's multi_logloss: 0.450379	valid_0's f1_score: 0.979871
[9]	valid_0's multi_logloss: 0.408513	valid_0's f1_score: 0.979884
[10]	valid_0's multi_logloss: 0.37126	valid_0's f1_score: 0.980461
[11]	valid_0's multi_logloss: 0.3381	valid_0's f1_score: 0.981251
[12]	valid_0's multi_logloss: 0.308454	valid_0's f1_score: 0.981535
[13]	valid_0's multi_logloss: 0.282083	valid_0's f1_score: 0.981759
[14]	valid_0's multi_logloss: 0.258294	valid_0's f1_score: 0.98188

[120]	valid_0's multi_logloss: 0.0165852	valid_0's f1_score: 0.994513
[121]	valid_0's multi_logloss: 0.0165228	valid_0's f1_score: 0.994527
[122]	valid_0's multi_logloss: 0.0164775	valid_0's f1_score: 0.994513
[123]	valid_0's multi_logloss: 0.0164108	valid_0's f1_score: 0.994556
[124]	valid_0's multi_logloss: 0.0163499	valid_0's f1_score: 0.994542
[125]	valid_0's multi_logloss: 0.0162927	valid_0's f1_score: 0.994586
[126]	valid_0's multi_logloss: 0.0162356	valid_0's f1_score: 0.994557
[127]	valid_0's multi_logloss: 0.0161557	valid_0's f1_score: 0.994557
[128]	valid_0's multi_logloss: 0.0161013	valid_0's f1_score: 0.994571
[129]	valid_0's multi_logloss: 0.0160686	valid_0's f1_score: 0.994585
[130]	valid_0's multi_logloss: 0.0160016	valid_0's f1_score: 0.994673
[131]	valid_0's multi_logloss: 0.0159468	valid_0's f1_score: 0.994658
[132]	valid_0's multi_logloss: 0.0159063	valid_0's f1_score: 0.994701
[133]	valid_0's multi_logloss: 0.015859	valid_0's f1_score: 0.994687
[134]	valid_0's multi

...

[28]	valid_0's multi_logloss: 0.0900303	valid_0's f1_score: 0.9857
[29]	valid_0's multi_logloss: 0.0846585	valid_0's f1_score: 0.986053
[30]	valid_0's multi_logloss: 0.0797034	valid_0's f1_score: 0.98664
[31]	valid_0's multi_logloss: 0.0751835	valid_0's f1_score: 0.986665
[32]	valid_0's multi_logloss: 0.0710043	valid_0's f1_score: 0.986932
[33]	valid_0's multi_logloss: 0.0671985	valid_0's f1_score: 0.987319
[34]	valid_0's multi_logloss: 0.0637501	valid_0's f1_score: 0.987704
[35]	valid_0's multi_logloss: 0.0605358	valid_0's f1_score: 0.987867
[36]	valid_0's multi_logloss: 0.0577134	valid_0's f1_score: 0.988117
[37]	valid_0's multi_logloss: 0.0550139	valid_0's f1_score: 0.988461
[38]	valid_0's multi_logloss: 0.0526405	valid_0's f1_score: 0.988649
[39]	valid_0's multi_logloss: 0.0504368	valid_0's f1_score: 0.988534
[40]	valid_0's multi_logloss: 0.0482989	valid_0's f1_score: 0.988872
[41]	valid_0's multi_logloss: 0.0464874	valid_0's f1_score: 0.989126
[42]	valid_0's multi_logloss: 0.04461

[147]	valid_0's multi_logloss: 0.0148814	valid_0's f1_score: 0.994854
[148]	valid_0's multi_logloss: 0.014838	valid_0's f1_score: 0.994911
[149]	valid_0's multi_logloss: 0.0147951	valid_0's f1_score: 0.994926
[150]	valid_0's multi_logloss: 0.0147598	valid_0's f1_score: 0.994939
[151]	valid_0's multi_logloss: 0.014721	valid_0's f1_score: 0.994954
[152]	valid_0's multi_logloss: 0.0146713	valid_0's f1_score: 0.994925
[153]	valid_0's multi_logloss: 0.0146478	valid_0's f1_score: 0.994896
[154]	valid_0's multi_logloss: 0.0145842	valid_0's f1_score: 0.994954
[155]	valid_0's multi_logloss: 0.0145525	valid_0's f1_score: 0.994969
[156]	valid_0's multi_logloss: 0.0145111	valid_0's f1_score: 0.994997
[157]	valid_0's multi_logloss: 0.0144906	valid_0's f1_score: 0.995012
[158]	valid_0's multi_logloss: 0.0144424	valid_0's f1_score: 0.995011
[159]	valid_0's multi_logloss: 0.0143988	valid_0's f1_score: 0.99504
[160]	valid_0's multi_logloss: 0.0143619	valid_0's f1_score: 0.995011
[161]	valid_0's multi_l

[268]	valid_0's multi_logloss: 0.0125472	valid_0's f1_score: 0.9955
[269]	valid_0's multi_logloss: 0.0125302	valid_0's f1_score: 0.995471
[270]	valid_0's multi_logloss: 0.0125203	valid_0's f1_score: 0.995486
[271]	valid_0's multi_logloss: 0.0125148	valid_0's f1_score: 0.995486
[272]	valid_0's multi_logloss: 0.0125032	valid_0's f1_score: 0.9955
[273]	valid_0's multi_logloss: 0.0124933	valid_0's f1_score: 0.995529
[274]	valid_0's multi_logloss: 0.0124868	valid_0's f1_score: 0.995514
[275]	valid_0's multi_logloss: 0.0124807	valid_0's f1_score: 0.995529
[276]	valid_0's multi_logloss: 0.0124713	valid_0's f1_score: 0.995529
[277]	valid_0's multi_logloss: 0.012452	valid_0's f1_score: 0.995572
[278]	valid_0's multi_logloss: 0.0124397	valid_0's f1_score: 0.995587
[279]	valid_0's multi_logloss: 0.0124297	valid_0's f1_score: 0.995572
[280]	valid_0's multi_logloss: 0.0124227	valid_0's f1_score: 0.995587
[281]	valid_0's multi_logloss: 0.0124187	valid_0's f1_score: 0.995572
[282]	valid_0's multi_log

[386]	valid_0's multi_logloss: 0.01176	valid_0's f1_score: 0.99573
[387]	valid_0's multi_logloss: 0.0117584	valid_0's f1_score: 0.995773
[388]	valid_0's multi_logloss: 0.0117551	valid_0's f1_score: 0.995759
[389]	valid_0's multi_logloss: 0.0117473	valid_0's f1_score: 0.995759
[390]	valid_0's multi_logloss: 0.0117503	valid_0's f1_score: 0.99573
[391]	valid_0's multi_logloss: 0.0117457	valid_0's f1_score: 0.99573
[392]	valid_0's multi_logloss: 0.0117391	valid_0's f1_score: 0.995744
[393]	valid_0's multi_logloss: 0.0117358	valid_0's f1_score: 0.995744
[394]	valid_0's multi_logloss: 0.0117302	valid_0's f1_score: 0.995744
[395]	valid_0's multi_logloss: 0.0117237	valid_0's f1_score: 0.995744
[396]	valid_0's multi_logloss: 0.011718	valid_0's f1_score: 0.995744
[397]	valid_0's multi_logloss: 0.0117124	valid_0's f1_score: 0.995744
[398]	valid_0's multi_logloss: 0.011711	valid_0's f1_score: 0.995744
[399]	valid_0's multi_logloss: 0.0117067	valid_0's f1_score: 0.995744
[400]	valid_0's multi_loglo

[507]	valid_0's multi_logloss: 0.0113779	valid_0's f1_score: 0.995816
[508]	valid_0's multi_logloss: 0.0113742	valid_0's f1_score: 0.995831
[509]	valid_0's multi_logloss: 0.0113742	valid_0's f1_score: 0.995802
[510]	valid_0's multi_logloss: 0.0113663	valid_0's f1_score: 0.995831
[511]	valid_0's multi_logloss: 0.0113682	valid_0's f1_score: 0.99586
[512]	valid_0's multi_logloss: 0.011368	valid_0's f1_score: 0.995859
[513]	valid_0's multi_logloss: 0.0113654	valid_0's f1_score: 0.995859
[514]	valid_0's multi_logloss: 0.0113606	valid_0's f1_score: 0.995874
[515]	valid_0's multi_logloss: 0.0113561	valid_0's f1_score: 0.995859
[516]	valid_0's multi_logloss: 0.0113544	valid_0's f1_score: 0.995873
[517]	valid_0's multi_logloss: 0.0113505	valid_0's f1_score: 0.995873
[518]	valid_0's multi_logloss: 0.0113504	valid_0's f1_score: 0.995873
Early stopping, best iteration is:
[468]	valid_0's multi_logloss: 0.011486	valid_0's f1_score: 0.995874
([0.9951663602198793, 0.9960700420027374, 0.99587627383450

In [16]:
# train.current_sevice.value_counts()
y.value_counts()

0    287219
1     38096
2     22037
Name: current_service, dtype: int64

In [18]:
# 导入数据包
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 基础配置信息
n_splits = 5
seed = 42

# lgb 参数
params={
    "learning_rate":0.1,
    "lambda_l1":0.1,
    "lambda_l2":0.2,
    "max_depth":4,
    "objective":"multiclass",
    "num_class":8,
    "silent":True,
}

# 读取数据
train = pd.read_csv('train/train_all.csv')
test = pd.read_csv('test/republish_test.csv')

train = train.loc[train.service_type == 4].reset_index(drop = True)
test = test.loc[test.service_type == 4].reset_index(drop = True)

'''
简单分析数据：
user_id 为编码后的数据，大小：
train data shape (612652, 27)
train data of user_id shape 612652
简单的1个用户1条样本的题目,标签的范围 current_service
'''
print('标签',set(train.columns)-set(test.columns))

print('train data shape',train.shape)
print('train data of user_id shape',len(set(train['user_id'])))
print('train data of current_service shape',(set(train['current_service'])))

print('train data shape',test.shape)
print('train data of user_id shape',len(set(test['user_id'])))

# 对标签编码 映射关系
label2current_service = dict(zip(range(0,len(set(train['current_service']))),sorted(list(set(train['current_service'])))))
current_service2label = dict(zip(sorted(list(set(train['current_service']))),range(0,len(set(train['current_service'])))))

# 原始数据的标签映射
train['current_service'] = train['current_service'].map(current_service2label)

# 构造原始数据
y = train.pop('current_service')
train_id = train.pop('user_id')
# 这个字段有点问题
X = train
train_col = train.columns

X_test = test[train_col]
test_id = test['user_id']

# 数据有问题数据
for i in train_col:
    X[i] = X[i].replace("\\N",-1)
    X_test[i] = X_test[i].replace("\\N",-1)

X,y,X_test = X.values,y,X_test.values

# 采取k折模型方案
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# 自定义F1评价函数
def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(8, -1), axis=0)
    score_vali = f1_score(y_true=labels, y_pred=preds, average='weighted')
    return 'f1_score', score_vali, True

xx_score = []
cv_pred = []

skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
for index,(train_index,test_index) in enumerate(skf.split(X,y)):
    print(index)

    X_train,X_valid,y_train,y_valid = X[train_index],X[test_index],y[train_index],y[test_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    validation_data = lgb.Dataset(X_valid, label=y_valid)
#     print train_data.get_label().value_counts()
#     print validation_data.get_label().value_counts()
    clf=lgb.train(params,train_data,num_boost_round=100000,
                  valid_sets=[validation_data],early_stopping_rounds=50,
                  feval=f1_score_vali,verbose_eval=1)

    xx_pred = clf.predict(X_valid,num_iteration=clf.best_iteration)

    xx_pred = [np.argmax(x) for x in xx_pred]

    xx_score.append(f1_score(y_valid,xx_pred,average='weighted'))

    y_test = clf.predict(X_test,num_iteration=clf.best_iteration)

    y_test = [np.argmax(x) for x in y_test]

    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))

# 投票
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))

# 保存结果
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = submit
df_test['predict'] = df_test['predict'].map(label2current_service)

df_test.to_csv('sub/sub4.csv',index=False)

print(xx_score,np.mean(xx_score))

('\xe6\xa0\x87\xe7\xad\xbe', set(['current_service']))
('train data shape', (396638, 27))
('train data of user_id shape', 396638)
('train data of current_service shape', set([99999830, 99999825, 99999826, 99999827, 99999828, 89950166, 89950167, 89950168]))
('train data shape', (67163, 26))
('train data of user_id shape', 67163)
0
[1]	valid_0's multi_logloss: 1.89528	valid_0's f1_score: 0.691776
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_logloss: 1.75906	valid_0's f1_score: 0.705836
[3]	valid_0's multi_logloss: 1.64668	valid_0's f1_score: 0.703506
[4]	valid_0's multi_logloss: 1.55187	valid_0's f1_score: 0.719954
[5]	valid_0's multi_logloss: 1.47037	valid_0's f1_score: 0.726432
[6]	valid_0's multi_logloss: 1.39859	valid_0's f1_score: 0.729299
[7]	valid_0's multi_logloss: 1.33503	valid_0's f1_score: 0.73396
[8]	valid_0's multi_logloss: 1.27809	valid_0's f1_score: 0.738194
[9]	valid_0's multi_logloss: 1.22657	valid_0's f1_score: 0.740282
[10]	valid_0'

[117]	valid_0's multi_logloss: 0.475606	valid_0's f1_score: 0.833526
[118]	valid_0's multi_logloss: 0.47468	valid_0's f1_score: 0.833767
[119]	valid_0's multi_logloss: 0.47376	valid_0's f1_score: 0.833849
[120]	valid_0's multi_logloss: 0.472655	valid_0's f1_score: 0.834362
[121]	valid_0's multi_logloss: 0.471844	valid_0's f1_score: 0.834532
[122]	valid_0's multi_logloss: 0.470953	valid_0's f1_score: 0.834728
[123]	valid_0's multi_logloss: 0.469752	valid_0's f1_score: 0.835009
[124]	valid_0's multi_logloss: 0.468987	valid_0's f1_score: 0.834957
[125]	valid_0's multi_logloss: 0.468081	valid_0's f1_score: 0.835465
[126]	valid_0's multi_logloss: 0.467259	valid_0's f1_score: 0.835618
[127]	valid_0's multi_logloss: 0.46647	valid_0's f1_score: 0.835942
[128]	valid_0's multi_logloss: 0.46572	valid_0's f1_score: 0.836375
[129]	valid_0's multi_logloss: 0.46496	valid_0's f1_score: 0.836452
[130]	valid_0's multi_logloss: 0.464292	valid_0's f1_score: 0.836851
[131]	valid_0's multi_logloss: 0.463758

[237]	valid_0's multi_logloss: 0.410118	valid_0's f1_score: 0.85281
[238]	valid_0's multi_logloss: 0.409882	valid_0's f1_score: 0.852766
[239]	valid_0's multi_logloss: 0.409654	valid_0's f1_score: 0.852805
[240]	valid_0's multi_logloss: 0.409383	valid_0's f1_score: 0.852913
[241]	valid_0's multi_logloss: 0.409056	valid_0's f1_score: 0.853128
[242]	valid_0's multi_logloss: 0.408859	valid_0's f1_score: 0.853286
[243]	valid_0's multi_logloss: 0.408542	valid_0's f1_score: 0.853382
[244]	valid_0's multi_logloss: 0.408313	valid_0's f1_score: 0.853456
[245]	valid_0's multi_logloss: 0.407989	valid_0's f1_score: 0.853541
[246]	valid_0's multi_logloss: 0.407746	valid_0's f1_score: 0.853546
[247]	valid_0's multi_logloss: 0.40742	valid_0's f1_score: 0.853563
[248]	valid_0's multi_logloss: 0.4071	valid_0's f1_score: 0.853739
[249]	valid_0's multi_logloss: 0.406852	valid_0's f1_score: 0.853704
[250]	valid_0's multi_logloss: 0.406506	valid_0's f1_score: 0.853903
[251]	valid_0's multi_logloss: 0.40628

[358]	valid_0's multi_logloss: 0.383819	valid_0's f1_score: 0.86038
[359]	valid_0's multi_logloss: 0.383694	valid_0's f1_score: 0.86034
[360]	valid_0's multi_logloss: 0.38346	valid_0's f1_score: 0.860531
[361]	valid_0's multi_logloss: 0.383187	valid_0's f1_score: 0.860516
[362]	valid_0's multi_logloss: 0.382967	valid_0's f1_score: 0.860643
[363]	valid_0's multi_logloss: 0.382706	valid_0's f1_score: 0.86066
[364]	valid_0's multi_logloss: 0.382587	valid_0's f1_score: 0.860697
[365]	valid_0's multi_logloss: 0.382371	valid_0's f1_score: 0.860723
[366]	valid_0's multi_logloss: 0.382052	valid_0's f1_score: 0.860853
[367]	valid_0's multi_logloss: 0.38178	valid_0's f1_score: 0.860945
[368]	valid_0's multi_logloss: 0.381603	valid_0's f1_score: 0.860952
[369]	valid_0's multi_logloss: 0.381423	valid_0's f1_score: 0.861118
[370]	valid_0's multi_logloss: 0.381293	valid_0's f1_score: 0.861194
[371]	valid_0's multi_logloss: 0.381167	valid_0's f1_score: 0.861086
[372]	valid_0's multi_logloss: 0.381025

[479]	valid_0's multi_logloss: 0.367131	valid_0's f1_score: 0.865877
[480]	valid_0's multi_logloss: 0.367058	valid_0's f1_score: 0.865886
[481]	valid_0's multi_logloss: 0.36698	valid_0's f1_score: 0.865985
[482]	valid_0's multi_logloss: 0.366885	valid_0's f1_score: 0.866038
[483]	valid_0's multi_logloss: 0.36682	valid_0's f1_score: 0.866092
[484]	valid_0's multi_logloss: 0.366729	valid_0's f1_score: 0.866106
[485]	valid_0's multi_logloss: 0.366677	valid_0's f1_score: 0.866172
[486]	valid_0's multi_logloss: 0.366588	valid_0's f1_score: 0.866149
[487]	valid_0's multi_logloss: 0.366478	valid_0's f1_score: 0.866263
[488]	valid_0's multi_logloss: 0.366442	valid_0's f1_score: 0.866326
[489]	valid_0's multi_logloss: 0.366347	valid_0's f1_score: 0.866291
[490]	valid_0's multi_logloss: 0.366279	valid_0's f1_score: 0.866341
[491]	valid_0's multi_logloss: 0.366165	valid_0's f1_score: 0.866245
[492]	valid_0's multi_logloss: 0.366081	valid_0's f1_score: 0.8663
[493]	valid_0's multi_logloss: 0.36602

...

[1749]	valid_0's multi_logloss: 0.315776	valid_0's f1_score: 0.884648
[1750]	valid_0's multi_logloss: 0.315763	valid_0's f1_score: 0.884674
[1751]	valid_0's multi_logloss: 0.315748	valid_0's f1_score: 0.884592
[1752]	valid_0's multi_logloss: 0.31572	valid_0's f1_score: 0.884609
[1753]	valid_0's multi_logloss: 0.315703	valid_0's f1_score: 0.884632
[1754]	valid_0's multi_logloss: 0.315678	valid_0's f1_score: 0.884633
[1755]	valid_0's multi_logloss: 0.315678	valid_0's f1_score: 0.884594
[1756]	valid_0's multi_logloss: 0.315661	valid_0's f1_score: 0.884606
[1757]	valid_0's multi_logloss: 0.315651	valid_0's f1_score: 0.884533
[1758]	valid_0's multi_logloss: 0.315639	valid_0's f1_score: 0.884457
[1759]	valid_0's multi_logloss: 0.315626	valid_0's f1_score: 0.884518
[1760]	valid_0's multi_logloss: 0.31561	valid_0's f1_score: 0.88462
[1761]	valid_0's multi_logloss: 0.315602	valid_0's f1_score: 0.884541
[1762]	valid_0's multi_logloss: 0.315595	valid_0's f1_score: 0.884518
[1763]	valid_0's multi_

[1867]	valid_0's multi_logloss: 0.314222	valid_0's f1_score: 0.885107
[1868]	valid_0's multi_logloss: 0.314204	valid_0's f1_score: 0.885049
[1869]	valid_0's multi_logloss: 0.314164	valid_0's f1_score: 0.884991
[1870]	valid_0's multi_logloss: 0.314157	valid_0's f1_score: 0.88502
[1871]	valid_0's multi_logloss: 0.314148	valid_0's f1_score: 0.88507
[1872]	valid_0's multi_logloss: 0.314143	valid_0's f1_score: 0.885
[1873]	valid_0's multi_logloss: 0.314134	valid_0's f1_score: 0.884974
[1874]	valid_0's multi_logloss: 0.314121	valid_0's f1_score: 0.88496
[1875]	valid_0's multi_logloss: 0.314109	valid_0's f1_score: 0.885059
[1876]	valid_0's multi_logloss: 0.314093	valid_0's f1_score: 0.885087
[1877]	valid_0's multi_logloss: 0.31408	valid_0's f1_score: 0.885065
[1878]	valid_0's multi_logloss: 0.314069	valid_0's f1_score: 0.885105
[1879]	valid_0's multi_logloss: 0.314049	valid_0's f1_score: 0.885101
[1880]	valid_0's multi_logloss: 0.314036	valid_0's f1_score: 0.885129
[1881]	valid_0's multi_logl

[1985]	valid_0's multi_logloss: 0.312647	valid_0's f1_score: 0.885868
[1986]	valid_0's multi_logloss: 0.312637	valid_0's f1_score: 0.885868
[1987]	valid_0's multi_logloss: 0.312622	valid_0's f1_score: 0.885794
[1988]	valid_0's multi_logloss: 0.312611	valid_0's f1_score: 0.885821
[1989]	valid_0's multi_logloss: 0.312591	valid_0's f1_score: 0.885844
[1990]	valid_0's multi_logloss: 0.312573	valid_0's f1_score: 0.885815
[1991]	valid_0's multi_logloss: 0.312563	valid_0's f1_score: 0.885817
[1992]	valid_0's multi_logloss: 0.312553	valid_0's f1_score: 0.885905
[1993]	valid_0's multi_logloss: 0.312539	valid_0's f1_score: 0.885921
[1994]	valid_0's multi_logloss: 0.312522	valid_0's f1_score: 0.885907
[1995]	valid_0's multi_logloss: 0.312506	valid_0's f1_score: 0.885884
[1996]	valid_0's multi_logloss: 0.312499	valid_0's f1_score: 0.885893
[1997]	valid_0's multi_logloss: 0.31249	valid_0's f1_score: 0.885894
[1998]	valid_0's multi_logloss: 0.312486	valid_0's f1_score: 0.885881
[1999]	valid_0's mult

[2103]	valid_0's multi_logloss: 0.311136	valid_0's f1_score: 0.88657
[2104]	valid_0's multi_logloss: 0.311124	valid_0's f1_score: 0.886624
[2105]	valid_0's multi_logloss: 0.311118	valid_0's f1_score: 0.886573
[2106]	valid_0's multi_logloss: 0.311099	valid_0's f1_score: 0.886665
[2107]	valid_0's multi_logloss: 0.3111	valid_0's f1_score: 0.886648
[2108]	valid_0's multi_logloss: 0.31109	valid_0's f1_score: 0.886645
[2109]	valid_0's multi_logloss: 0.311075	valid_0's f1_score: 0.886619
[2110]	valid_0's multi_logloss: 0.31107	valid_0's f1_score: 0.886604
[2111]	valid_0's multi_logloss: 0.311062	valid_0's f1_score: 0.886606
[2112]	valid_0's multi_logloss: 0.311041	valid_0's f1_score: 0.886632
[2113]	valid_0's multi_logloss: 0.311024	valid_0's f1_score: 0.886621
[2114]	valid_0's multi_logloss: 0.311021	valid_0's f1_score: 0.886621
[2115]	valid_0's multi_logloss: 0.311021	valid_0's f1_score: 0.886609
[2116]	valid_0's multi_logloss: 0.311005	valid_0's f1_score: 0.886545
[2117]	valid_0's multi_lo

[2221]	valid_0's multi_logloss: 0.309872	valid_0's f1_score: 0.886961
[2222]	valid_0's multi_logloss: 0.309867	valid_0's f1_score: 0.886984
[2223]	valid_0's multi_logloss: 0.30986	valid_0's f1_score: 0.887047
[2224]	valid_0's multi_logloss: 0.309837	valid_0's f1_score: 0.887058
[2225]	valid_0's multi_logloss: 0.309831	valid_0's f1_score: 0.886999
[2226]	valid_0's multi_logloss: 0.309833	valid_0's f1_score: 0.886961
[2227]	valid_0's multi_logloss: 0.309822	valid_0's f1_score: 0.887025
[2228]	valid_0's multi_logloss: 0.309804	valid_0's f1_score: 0.887021
[2229]	valid_0's multi_logloss: 0.309788	valid_0's f1_score: 0.887114
[2230]	valid_0's multi_logloss: 0.309772	valid_0's f1_score: 0.887074
[2231]	valid_0's multi_logloss: 0.309766	valid_0's f1_score: 0.887112
[2232]	valid_0's multi_logloss: 0.309758	valid_0's f1_score: 0.887162
[2233]	valid_0's multi_logloss: 0.309741	valid_0's f1_score: 0.88721
[2234]	valid_0's multi_logloss: 0.309744	valid_0's f1_score: 0.887148
[2235]	valid_0's multi

In [20]:
test = pd.read_csv('test/republish_test.csv')
test = test.loc[test.service_type == 3]
test_id = test['user_id']
df_test = pd.DataFrame()
df_test['id'] = list(test_id.unique())
df_test['predict'] = 99104722
df_test.to_csv('sub/sub3.csv',index = False)

In [23]:
res1 = pd.read_csv('sub/sub1.csv')
res3 = pd.read_csv('sub/sub3.csv')
res4 = pd.read_csv('sub/sub4.csv')
res = pd.concat([res1,res3,res4])
res.to_csv('sub/sub.csv',index = False)
res.predict.value_counts()

90063345    116009
89950167     20468
89950166     17207
90109916      9901
99999828      8692
90155946      6908
99999826      5425
99999827      4318
99999830      4090
89950168      3743
99999825      3220
99104722        19
Name: predict, dtype: int64